<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [4]:
# Import any packages you need here
# Also define any variables as needed
import copy

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    # initialization
    def __init__(self, state, gval=0, fval=0, parent=None):                    
        self.state = state
        self.parent = parent
        self.pruned = False # flag for pruning
         
        self.G = gval # dist from start node
        self.F = fval # g+h
        
        
        self.dims = (len(state)-1, len(state[0])-1)
    
    # for comparison in priority queue
    def __lt__(self,other):
        return self.F < other.F

    # equality is defined by the same state
    def __eq__(self, other):
        return self.__class__ == other.__class__ and self.state == other.state
    
    # pretty print state, for debugging and display
    def __str__(self):
        res = "_____\n"
        for row in self.state:
            res += ' '.join(map(str, row))
            res += '\n'
        res += "_____\n"
        return res
    
    def next_states(self):
        # find the empty space
        zr = zc = -1
        for i, row in enumerate(self.state):
            try:
                zr, zc = i, row.index(0)
            except ValueError:
                pass
        
        nextStates = []
        
        # make children with one move diff        
        if zr > 0:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr-1][zc] = state[zr-1][zc], state[zr][zc]
            nextStates.append(state)
        if zr < self.dims[0]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr+1][zc] = state[zr+1][zc], state[zr][zc]
            nextStates.append(state)
        if zc > 0 :
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc-1] = state[zr][zc-1], state[zr][zc]
            nextStates.append(state)
        if zc < self.dims[1]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc+1] = state[zr][zc+1], state[zr][zc] 
            nextStates.append(state)
        return(nextStates)
    


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [5]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    misp = 0
    
    for r, row in enumerate(state):
        for c, t in enumerate(row):
            if t != 0 and t!= r*3+c:
                misp += 1
    return(misp)

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    manh = 0
    
    for r, row in enumerate(state):
        for c, t in enumerate(row):
            if t != 0:
                manh += abs(t//3-r) # row distance
                manh += abs(t%3-c) # col distance
    return(manh)
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]


<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [27]:
# creating child nodes for valid next moves

# Import any packages or define any helper functions you need here

from queue import PriorityQueue
from functools import reduce

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    # _____ HELPER _____
    def validate_start_state(state):
        n = len(state)
        
        # dimension n x n
        if n == 0 or type(state[0]) != list:
            return(-1)
        elif n != len(state[0]):
            return(-1)
        
        # contains every number from 0 to n**2-1
        seen_ind = [False] * n**2 # each index represents if the index has been seen
        
        for row in state:
            for t in row:
                seen_ind[t] = True 
        
        if sum(seen_ind) != n**2: # shortcut: returns whether every index is seen
            return(-1)
            
        # EXTENSION: solvability
        '''
        Let inversion be defined as the number of "out of order" pairs
        Ex: An (A,B) pair is inverted if the value of A is more than B,
            but when the state is flattened into a single list
            the index of A is lower than the index of B.
        
        Reasoning for using inversions:
        Let n be the width of the board
        An n-puzzle only has two types of moves: a horizontal or vertical swap
        A horizontal swap does not change the number of inversions in the state
        A vertical swap will change the inversions on the tiles between its start 
        state and end state by exactly 1 (the tile and itself form one pair), 
        thus it will change the number of inversions by exactly (n-1).
        
        For any odd numbered n (such as 9), (n-1) will be even.
        Thus, a start state is solvable iff it has an even number of inversions.

        '''
        
        # count inversions:
        flat = reduce(lambda z, y :z + y, state)  
        num_inv = 0
        for i in range(n):                 
            for j in range(i + 1, n):
                if (flat[i] != 0 and flat[j] != 0 and flat[i] > flat[j]): 
                    num_inv += 1                    
        if n%2 == 1 and num_inv %2 == 1:
            return(-2)
        
        return(0) 
        
            
    def create_goal(state):
        v = 0
        goal = []
        for r, row in enumerate(state):
            arow = []
            for c, _ in enumerate(row):
                arow.append(v)
                v += 1
            goal.append(arow)
        return(goal)
    
    def trace_path(node, path = {}, height = 0):
        if node.parent == None:
            return(path)
        else:
            path[height] = node.state
            return(trace_path(node.parent, path, height + 1))
    
    def immortalize(lst_of_lst):
        return(tuple(tuple(x) for x in lst_of_lst))

    # _____ VALIDATE INPUT _____
    err = validate_start_state(state)
    if err != 0:
        return(0,0,0,None,err)
    
    # _____ INIT _____
    steps = exp = max_frontier = 0
    opt_path = [state]
    goal = create_goal(state)
    
    # create start node
    start_node = PuzzleNode(state)
    start_node.F = heuristic(state)
        
    # priority queue
    frontier = PriorityQueue()
    frontier.put(start_node) # excluding the g(s) = 0
    
    # lookup for best paths found to a state
    clookup = {}
    clookup[immortalize(state)] = start_node
        
    while not frontier.empty():
        max_frontier = max(max_frontier, frontier.qsize())
        cur_node = frontier.get()
        
        if cur_node.pruned:
            continue # skip if pruned
            
        if cur_node.state == goal:
            break
        
        for state in cur_node.next_states():
            # cost = dist from start + heuristic
            G = cur_node.G + 1
            H = heuristic(state)
            
            imState = immortalize(state)

            if imState in clookup:
                if clookup[imState].G <= G:
                    continue # if better path to this state is known, ignore this path
                else:
                    clookup[imState].pruned = True # else, prune old path
            
            # add new node to frontier, update lookup
            child_node = PuzzleNode(state, G, G + heuristic(state), cur_node)
            frontier.put(child_node)
            clookup[imState] = child_node   
                
        exp += 1
    
    # recreate optimal path
    while cur_node.parent is not None:
        opt_path.insert(1, cur_node.state)
        cur_node = cur_node.parent
        steps += 1
    return(steps, exp, max_frontier, opt_path, err)


In [28]:
solvePuzzle([[7,0,8],[4,6,1],[5,3,2]], heuristics[0])

[7, 0, 8, 4, 6, 1, 5, 3, 2]


(25,
 25349,
 12075,
 [[[7, 0, 8], [4, 6, 1], [5, 3, 2]],
  [[7, 6, 8], [4, 0, 1], [5, 3, 2]],
  [[7, 6, 8], [4, 3, 1], [5, 0, 2]],
  [[7, 6, 8], [4, 3, 1], [0, 5, 2]],
  [[7, 6, 8], [0, 3, 1], [4, 5, 2]],
  [[7, 6, 8], [3, 0, 1], [4, 5, 2]],
  [[7, 0, 8], [3, 6, 1], [4, 5, 2]],
  [[7, 8, 0], [3, 6, 1], [4, 5, 2]],
  [[7, 8, 1], [3, 6, 0], [4, 5, 2]],
  [[7, 8, 1], [3, 6, 2], [4, 5, 0]],
  [[7, 8, 1], [3, 6, 2], [4, 0, 5]],
  [[7, 8, 1], [3, 0, 2], [4, 6, 5]],
  [[7, 0, 1], [3, 8, 2], [4, 6, 5]],
  [[0, 7, 1], [3, 8, 2], [4, 6, 5]],
  [[3, 7, 1], [0, 8, 2], [4, 6, 5]],
  [[3, 7, 1], [4, 8, 2], [0, 6, 5]],
  [[3, 7, 1], [4, 8, 2], [6, 0, 5]],
  [[3, 7, 1], [4, 0, 2], [6, 8, 5]],
  [[3, 0, 1], [4, 7, 2], [6, 8, 5]],
  [[3, 1, 0], [4, 7, 2], [6, 8, 5]],
  [[3, 1, 2], [4, 7, 0], [6, 8, 5]],
  [[3, 1, 2], [4, 7, 5], [6, 8, 0]],
  [[3, 1, 2], [4, 7, 5], [6, 0, 8]],
  [[3, 1, 2], [4, 0, 5], [6, 7, 8]],
  [[3, 1, 2], [0, 4, 5], [6, 7, 8]],
  [[0, 1, 2], [3, 4, 5], [6, 7, 8]]],
 0)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [22]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [23]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [29]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


[2, 3, 7, 1, 8, 0, 6, 5, 4]
[2, 3, 7, 1, 8, 0, 6, 5, 4]
[7, 0, 8, 4, 6, 1, 5, 3, 2]
[7, 0, 8, 4, 6, 1, 5, 3, 2]
[5, 7, 6, 2, 4, 3, 8, 1, 0]
[5, 7, 6, 2, 4, 3, 8, 1, 0]


AssertionError: 

In [18]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

[1, 2, 6, 3, 0, 9, 5, 7, 4, 13, 10, 11, 8, 12, 14, 15]
4 1
0 [[1, 2, 6, 3], [0, 9, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 14
0 [[1, 2, 6, 3], [0, 9, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 14
0 [[1, 2, 6, 3], [0, 9, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 14
1 [[0, 2, 6, 3], [1, 9, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 15
2 [[1, 2, 6, 3], [4, 9, 5, 7], [0, 13, 10, 11], [8, 12, 14, 15]] 15
2 [[1, 2, 6, 3], [4, 9, 5, 7], [0, 13, 10, 11], [8, 12, 14, 15]] 15
3 [[1, 2, 6, 3], [9, 0, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 15
3 [[1, 2, 6, 3], [9, 0, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 15
3 [[1, 2, 6, 3], [9, 0, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 16
4 [[1, 2, 6, 3], [4, 9, 5, 7], [8, 13, 10, 11], [0, 12, 14, 15]] 16
5 [[1, 2, 6, 3], [4, 9, 5, 7], [13, 0, 10, 11], [8, 12, 14, 15]] 16
5 [[1, 2, 6, 3], [4, 9, 5, 7], [13, 0, 10, 11], [8, 12, 14, 15]] 16
5 [[1, 2, 6, 3], [4, 9, 5, 7], [13, 0, 10, 11], [8, 12, 14, 15]] 16
6 [[2, 0, 6, 3], [1, 9, 5, 7], [4, 13, 10, 11], [8, 12, 1

337 [[2, 6, 5, 3], [0, 1, 9, 7], [4, 13, 10, 11], [8, 12, 14, 15]] 21
338 [[1, 5, 2, 3], [0, 4, 6, 7], [13, 9, 10, 11], [8, 12, 14, 15]] 21
338 [[1, 5, 2, 3], [0, 4, 6, 7], [13, 9, 10, 11], [8, 12, 14, 15]] 21
339 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 13, 14, 10], [8, 12, 0, 15]] 21
339 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 13, 14, 10], [8, 12, 0, 15]] 21
340 [[1, 2, 6, 3], [9, 5, 0, 11], [4, 13, 7, 10], [8, 12, 14, 15]] 21
340 [[1, 2, 6, 3], [9, 5, 0, 11], [4, 13, 7, 10], [8, 12, 14, 15]] 20
340 [[1, 2, 6, 3], [9, 5, 0, 11], [4, 13, 7, 10], [8, 12, 14, 15]] 21
341 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 0, 13, 10], [8, 12, 14, 15]] 21
341 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 0, 13, 10], [8, 12, 14, 15]] 21
341 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 0, 13, 10], [8, 12, 14, 15]] 21
342 [[0, 1, 6, 3], [9, 2, 10, 7], [4, 5, 13, 11], [8, 12, 14, 15]] 21
343 [[1, 2, 6, 3], [4, 5, 10, 7], [13, 0, 9, 11], [8, 12, 14, 15]] 21
343 [[1, 2, 6, 3], [4, 5, 10, 7], [13, 0, 9, 11], [8, 12, 14, 15]] 21
343 [[1, 2, 6, 3], [

652 [[1, 2, 6, 3], [4, 9, 5, 7], [10, 0, 14, 11], [13, 8, 12, 15]] 22
652 [[1, 2, 6, 3], [4, 9, 5, 7], [10, 0, 14, 11], [13, 8, 12, 15]] 22
653 [[1, 0, 6, 3], [9, 2, 5, 11], [4, 13, 7, 10], [8, 12, 14, 15]] 21
653 [[1, 0, 6, 3], [9, 2, 5, 11], [4, 13, 7, 10], [8, 12, 14, 15]] 22
654 [[1, 2, 6, 3], [4, 5, 10, 7], [13, 12, 9, 11], [8, 0, 14, 15]] 22
654 [[1, 2, 6, 3], [4, 5, 10, 7], [13, 12, 9, 11], [8, 0, 14, 15]] 22
655 [[1, 2, 6, 3], [4, 9, 5, 7], [8, 13, 11, 15], [12, 14, 0, 10]] 21
655 [[1, 2, 6, 3], [4, 9, 5, 7], [8, 13, 11, 15], [12, 14, 0, 10]] 22
656 [[1, 2, 6, 3], [4, 0, 10, 7], [13, 5, 9, 11], [8, 12, 14, 15]] 22
656 [[1, 2, 6, 3], [4, 0, 10, 7], [13, 5, 9, 11], [8, 12, 14, 15]] 21
657 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 12, 13, 10], [8, 0, 14, 15]] 22
657 [[1, 2, 6, 3], [9, 5, 7, 11], [4, 12, 13, 10], [8, 0, 14, 15]] 22
658 [[9, 1, 6, 3], [4, 2, 5, 7], [8, 13, 10, 11], [12, 14, 15, 0]] 22
659 [[2, 6, 5, 3], [1, 9, 10, 7], [0, 4, 13, 11], [8, 12, 14, 15]] 22
659 [[2, 6, 5, 3], [

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [19]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

[7, 5, 6, 2, 4, 3, 8, 1, 0]
3 2
0 [[7, 5, 6], [2, 4, 3], [8, 1, 0]] 1
0 [[7, 5, 6], [2, 4, 3], [8, 1, 0]] 1
1 [[7, 5, 6], [2, 4, 0], [8, 1, 3]] 2
1 [[7, 5, 6], [2, 4, 0], [8, 1, 3]] 2
2 [[7, 5, 6], [2, 4, 3], [8, 0, 1]] 2
2 [[7, 5, 6], [2, 4, 3], [8, 0, 1]] 2
3 [[7, 5, 0], [2, 4, 6], [8, 1, 3]] 3
4 [[7, 5, 6], [2, 0, 3], [8, 4, 1]] 3
4 [[7, 5, 6], [2, 0, 3], [8, 4, 1]] 3
4 [[7, 5, 6], [2, 0, 3], [8, 4, 1]] 3
5 [[7, 5, 6], [2, 4, 3], [0, 8, 1]] 3
6 [[7, 5, 6], [2, 0, 4], [8, 1, 3]] 3
6 [[7, 5, 6], [2, 0, 4], [8, 1, 3]] 3
6 [[7, 5, 6], [2, 0, 4], [8, 1, 3]] 3
7 [[7, 0, 5], [2, 4, 6], [8, 1, 3]] 4
7 [[7, 0, 5], [2, 4, 6], [8, 1, 3]] 4
8 [[7, 5, 6], [0, 4, 3], [2, 8, 1]] 4
8 [[7, 5, 6], [0, 4, 3], [2, 8, 1]] 4
9 [[7, 5, 6], [2, 1, 4], [8, 0, 3]] 4
9 [[7, 5, 6], [2, 1, 4], [8, 0, 3]] 4
10 [[7, 5, 6], [0, 2, 4], [8, 1, 3]] 4
10 [[7, 5, 6], [0, 2, 4], [8, 1, 3]] 4
11 [[7, 0, 6], [2, 5, 4], [8, 1, 3]] 4
11 [[7, 0, 6], [2, 5, 4], [8, 1, 3]] 4
12 [[7, 5, 6], [0, 2, 3], [8, 4, 1]] 4
12 [[7, 5, 6]

404 [[2, 7, 6], [0, 8, 3], [4, 5, 1]] 10
405 [[2, 0, 6], [8, 7, 3], [4, 5, 1]] 10
405 [[2, 0, 6], [8, 7, 3], [4, 5, 1]] 10
406 [[7, 4, 5], [1, 6, 0], [2, 8, 3]] 10
406 [[7, 4, 5], [1, 6, 0], [2, 8, 3]] 10
407 [[5, 0, 6], [7, 8, 4], [1, 2, 3]] 10
407 [[5, 0, 6], [7, 8, 4], [1, 2, 3]] 10
408 [[5, 6, 4], [0, 7, 2], [8, 1, 3]] 10
408 [[5, 6, 4], [0, 7, 2], [8, 1, 3]] 10
409 [[7, 5, 6], [4, 1, 8], [2, 0, 3]] 10
409 [[7, 5, 6], [4, 1, 8], [2, 0, 3]] 10
410 [[7, 0, 6], [4, 5, 8], [2, 1, 3]] 10
410 [[7, 0, 6], [4, 5, 8], [2, 1, 3]] 10
411 [[8, 7, 6], [0, 5, 4], [1, 2, 3]] 10
411 [[8, 7, 6], [0, 5, 4], [1, 2, 3]] 10
412 [[7, 6, 4], [0, 2, 3], [8, 5, 1]] 10
412 [[7, 6, 4], [0, 2, 3], [8, 5, 1]] 10
413 [[7, 0, 4], [2, 6, 3], [8, 5, 1]] 10
413 [[7, 0, 4], [2, 6, 3], [8, 5, 1]] 10
414 [[2, 7, 5], [8, 4, 0], [1, 3, 6]] 10
414 [[2, 7, 5], [8, 4, 0], [1, 3, 6]] 10
415 [[7, 5, 6], [0, 4, 1], [2, 3, 8]] 10
415 [[7, 5, 6], [0, 4, 1], [2, 3, 8]] 10
416 [[7, 3, 5], [0, 4, 6], [2, 8, 1]] 10
417 [[7, 3, 5], 

825 [[5, 3, 6], [7, 1, 0], [2, 8, 4]] 12
826 [[6, 2, 3], [7, 5, 0], [8, 4, 1]] 12
826 [[6, 2, 3], [7, 5, 0], [8, 4, 1]] 12
827 [[8, 7, 6], [5, 2, 4], [1, 0, 3]] 12
827 [[8, 7, 6], [5, 2, 4], [1, 0, 3]] 12
828 [[8, 0, 6], [5, 7, 4], [1, 2, 3]] 12
828 [[8, 0, 6], [5, 7, 4], [1, 2, 3]] 12
829 [[7, 0, 3], [2, 4, 5], [8, 1, 6]] 12
829 [[7, 0, 3], [2, 4, 5], [8, 1, 6]] 12
830 [[7, 2, 5], [0, 8, 6], [4, 1, 3]] 12
830 [[7, 2, 5], [0, 8, 6], [4, 1, 3]] 12
831 [[7, 2, 5], [8, 1, 6], [4, 0, 3]] 12
831 [[7, 2, 5], [8, 1, 6], [4, 0, 3]] 12
832 [[6, 2, 3], [0, 7, 5], [8, 4, 1]] 12
832 [[6, 2, 3], [0, 7, 5], [8, 4, 1]] 12
833 [[7, 1, 4], [2, 5, 0], [8, 3, 6]] 12
833 [[7, 1, 4], [2, 5, 0], [8, 3, 6]] 12
834 [[5, 6, 4], [7, 3, 0], [8, 2, 1]] 12
834 [[5, 6, 4], [7, 3, 0], [8, 2, 1]] 12
835 [[2, 4, 7], [0, 5, 6], [8, 1, 3]] 12
835 [[2, 4, 7], [0, 5, 6], [8, 1, 3]] 12
836 [[2, 4, 7], [5, 1, 6], [8, 0, 3]] 12
836 [[2, 4, 7], [5, 1, 6], [8, 0, 3]] 12
837 [[2, 7, 5], [3, 4, 0], [8, 1, 6]] 12
837 [[2, 7, 5], 

1241 [[4, 5, 0], [7, 6, 3], [2, 8, 1]] 13
1242 [[0, 4, 6], [5, 3, 1], [7, 2, 8]] 13
1243 [[0, 7, 5], [2, 6, 3], [4, 8, 1]] 13
1244 [[7, 6, 2], [8, 0, 5], [1, 3, 4]] 13
1244 [[7, 6, 2], [8, 0, 5], [1, 3, 4]] 13
1244 [[7, 6, 2], [8, 0, 5], [1, 3, 4]] 13
1245 [[2, 7, 6], [4, 8, 5], [0, 1, 3]] 13
1246 [[2, 7, 3], [8, 0, 5], [4, 6, 1]] 13
1246 [[2, 7, 3], [8, 0, 5], [4, 6, 1]] 13
1246 [[2, 7, 3], [8, 0, 5], [4, 6, 1]] 13
1247 [[3, 2, 0], [7, 6, 5], [8, 4, 1]] 13
1248 [[0, 7, 1], [2, 6, 5], [8, 3, 4]] 13
1249 [[2, 7, 0], [6, 5, 4], [8, 1, 3]] 13
1250 [[7, 2, 4], [8, 0, 5], [1, 3, 6]] 13
1250 [[7, 2, 4], [8, 0, 5], [1, 3, 6]] 13
1250 [[7, 2, 4], [8, 0, 5], [1, 3, 6]] 13
1251 [[7, 5, 6], [8, 1, 4], [3, 2, 0]] 13
1252 [[0, 7, 6], [2, 3, 4], [8, 5, 1]] 13
1253 [[5, 6, 3], [2, 0, 1], [7, 8, 4]] 13
1253 [[5, 6, 3], [2, 0, 1], [7, 8, 4]] 13
1253 [[5, 6, 3], [2, 0, 1], [7, 8, 4]] 13
1254 [[4, 2, 5], [8, 0, 6], [1, 7, 3]] 13
1254 [[4, 2, 5], [8, 0, 6], [1, 7, 3]] 13
1254 [[4, 2, 5], [8, 0, 6], [1, 7,

1799 [[5, 8, 6], [7, 4, 3], [1, 2, 0]] 13
1800 [[4, 6, 3], [5, 7, 1], [2, 8, 0]] 13
1801 [[0, 6, 7], [2, 4, 5], [8, 1, 3]] 13
1802 [[7, 5, 6], [3, 0, 1], [8, 4, 2]] 13
1802 [[7, 5, 6], [3, 0, 1], [8, 4, 2]] 13
1802 [[7, 5, 6], [3, 0, 1], [8, 4, 2]] 13
1804 [[7, 5, 2], [8, 4, 6], [0, 1, 3]] 13
1805 [[7, 5, 3], [6, 0, 4], [2, 8, 1]] 13
1805 [[7, 5, 3], [6, 0, 4], [2, 8, 1]] 13
1806 [[7, 6, 8], [4, 0, 5], [2, 1, 3]] 13
1806 [[7, 6, 8], [4, 0, 5], [2, 1, 3]] 13
1807 [[7, 6, 0], [2, 8, 3], [4, 5, 1]] 13
1808 [[7, 3, 0], [8, 2, 5], [4, 1, 6]] 13
1809 [[8, 7, 6], [5, 2, 3], [0, 4, 1]] 13
1811 [[5, 4, 6], [7, 1, 8], [0, 2, 3]] 13
1812 [[7, 6, 3], [8, 0, 1], [4, 5, 2]] 13
1812 [[7, 6, 3], [8, 0, 1], [4, 5, 2]] 13
1812 [[7, 6, 3], [8, 0, 1], [4, 5, 2]] 13
1813 [[5, 8, 6], [1, 7, 4], [0, 2, 3]] 13
1814 [[7, 3, 5], [6, 0, 1], [2, 8, 4]] 13
1814 [[7, 3, 5], [6, 0, 1], [2, 8, 4]] 13
1814 [[7, 3, 5], [6, 0, 1], [2, 8, 4]] 13
1816 [[7, 1, 6], [2, 0, 4], [5, 8, 3]] 13
1816 [[7, 1, 6], [2, 0, 4], [5, 8,

...

33567 [[1, 2, 5], [3, 8, 0], [7, 4, 6]] 20
33568 [[7, 6, 5], [0, 8, 2], [4, 3, 1]] 20
33568 [[7, 6, 5], [0, 8, 2], [4, 3, 1]] 20
33569 [[7, 2, 4], [5, 1, 6], [8, 0, 3]] 20
33569 [[7, 2, 4], [5, 1, 6], [8, 0, 3]] 20
33570 [[3, 6, 5], [7, 2, 0], [4, 8, 1]] 20
33570 [[3, 6, 5], [7, 2, 0], [4, 8, 1]] 20
33571 [[6, 1, 5], [7, 2, 3], [4, 0, 8]] 20
33571 [[6, 1, 5], [7, 2, 3], [4, 0, 8]] 20
33572 [[3, 6, 8], [7, 2, 5], [4, 0, 1]] 20
33572 [[3, 6, 8], [7, 2, 5], [4, 0, 1]] 20
33573 [[3, 6, 5], [0, 2, 1], [7, 4, 8]] 20
33573 [[3, 6, 5], [0, 2, 1], [7, 4, 8]] 20
33574 [[5, 7, 6], [1, 3, 2], [8, 0, 4]] 20
33574 [[5, 7, 6], [1, 3, 2], [8, 0, 4]] 20
33575 [[5, 3, 1], [2, 7, 0], [8, 6, 4]] 20
33576 [[5, 6, 2], [0, 1, 7], [8, 3, 4]] 20
33577 [[5, 6, 2], [1, 3, 7], [8, 0, 4]] 20
33577 [[5, 6, 2], [1, 3, 7], [8, 0, 4]] 20
33578 [[5, 7, 6], [0, 1, 2], [8, 3, 4]] 20
33578 [[5, 7, 6], [0, 1, 2], [8, 3, 4]] 20
33579 [[8, 0, 6], [7, 2, 5], [3, 4, 1]] 20
33580 [[2, 0, 5], [4, 3, 7], [6, 8, 1]] 20
33580 [[2, 

34028 [[2, 0, 4], [8, 5, 6], [7, 3, 1]] 20
34029 [[2, 8, 7], [6, 3, 5], [1, 0, 4]] 20
34029 [[2, 8, 7], [6, 3, 5], [1, 0, 4]] 20
34030 [[6, 2, 7], [3, 5, 4], [8, 0, 1]] 20
34030 [[6, 2, 7], [3, 5, 4], [8, 0, 1]] 20
34031 [[6, 0, 7], [3, 2, 4], [8, 5, 1]] 20
34031 [[6, 0, 7], [3, 2, 4], [8, 5, 1]] 20
34033 [[2, 0, 7], [6, 8, 5], [1, 3, 4]] 20
34034 [[1, 5, 2], [0, 7, 4], [8, 3, 6]] 20
34034 [[1, 5, 2], [0, 7, 4], [8, 3, 6]] 20
34035 [[1, 5, 2], [7, 3, 4], [8, 0, 6]] 20
34035 [[1, 5, 2], [7, 3, 4], [8, 0, 6]] 20
34036 [[7, 5, 1], [8, 3, 0], [2, 4, 6]] 20
34036 [[7, 5, 1], [8, 3, 0], [2, 4, 6]] 20
34037 [[6, 0, 3], [2, 4, 1], [7, 8, 5]] 20
34038 [[2, 6, 1], [0, 4, 5], [7, 8, 3]] 20
34038 [[2, 6, 1], [0, 4, 5], [7, 8, 3]] 20
34039 [[7, 6, 1], [0, 3, 2], [4, 8, 5]] 20
34039 [[7, 6, 1], [0, 3, 2], [4, 8, 5]] 20
34040 [[5, 4, 1], [2, 6, 0], [7, 3, 8]] 20
34040 [[5, 4, 1], [2, 6, 0], [7, 3, 8]] 20
34041 [[3, 6, 4], [2, 7, 5], [8, 0, 1]] 20
34042 [[7, 0, 1], [3, 6, 2], [4, 8, 5]] 20
34042 [[7, 

34469 [[7, 5, 4], [0, 1, 8], [2, 6, 3]] 20
34470 [[1, 0, 6], [8, 7, 2], [5, 3, 4]] 20
34470 [[1, 0, 6], [8, 7, 2], [5, 3, 4]] 20
34471 [[2, 5, 6], [7, 8, 0], [3, 4, 1]] 20
34472 [[3, 4, 5], [0, 7, 6], [8, 1, 2]] 20
34472 [[3, 4, 5], [0, 7, 6], [8, 1, 2]] 20
34473 [[7, 0, 4], [1, 5, 8], [2, 6, 3]] 20
34473 [[7, 0, 4], [1, 5, 8], [2, 6, 3]] 20
34474 [[6, 3, 1], [7, 4, 5], [8, 0, 2]] 20
34475 [[6, 4, 5], [2, 3, 1], [7, 0, 8]] 20
34475 [[6, 4, 5], [2, 3, 1], [7, 0, 8]] 20
34476 [[5, 4, 1], [6, 2, 3], [7, 0, 8]] 20
34476 [[5, 4, 1], [6, 2, 3], [7, 0, 8]] 20
34477 [[5, 0, 1], [6, 4, 3], [7, 2, 8]] 20
34477 [[5, 0, 1], [6, 4, 3], [7, 2, 8]] 20
34478 [[6, 5, 3], [8, 7, 1], [4, 0, 2]] 20
34479 [[7, 1, 2], [8, 3, 0], [4, 6, 5]] 20
34480 [[7, 6, 4], [3, 2, 8], [1, 0, 5]] 20
34481 [[7, 0, 4], [3, 6, 8], [1, 2, 5]] 20
34481 [[7, 0, 4], [3, 6, 8], [1, 2, 5]] 20
34483 [[8, 7, 5], [0, 1, 3], [6, 4, 2]] 20
34484 [[8, 0, 5], [1, 7, 3], [6, 4, 2]] 20
34484 [[8, 0, 5], [1, 7, 3], [6, 4, 2]] 20
34485 [[8, 

34945 [[2, 6, 4], [0, 1, 7], [8, 3, 5]] 20
34946 [[8, 2, 4], [7, 3, 6], [1, 0, 5]] 20
34946 [[8, 2, 4], [7, 3, 6], [1, 0, 5]] 20
34947 [[8, 0, 4], [7, 2, 6], [1, 3, 5]] 20
34949 [[8, 2, 6], [1, 4, 5], [7, 0, 3]] 20
34950 [[8, 0, 6], [1, 2, 5], [7, 4, 3]] 20
34950 [[8, 0, 6], [1, 2, 5], [7, 4, 3]] 20
34951 [[6, 7, 4], [3, 5, 1], [2, 0, 8]] 20
34951 [[6, 7, 4], [3, 5, 1], [2, 0, 8]] 20
34952 [[5, 7, 6], [3, 4, 1], [2, 0, 8]] 20
34953 [[2, 6, 3], [8, 4, 0], [1, 5, 7]] 20
34954 [[8, 5, 6], [1, 7, 2], [4, 0, 3]] 20
34955 [[8, 0, 6], [1, 5, 2], [4, 7, 3]] 20
34956 [[2, 5, 6], [1, 7, 8], [3, 0, 4]] 20
34957 [[3, 2, 5], [0, 4, 1], [7, 6, 8]] 20
34957 [[3, 2, 5], [0, 4, 1], [7, 6, 8]] 20
34958 [[3, 0, 5], [4, 2, 1], [7, 6, 8]] 20
34958 [[3, 0, 5], [4, 2, 1], [7, 6, 8]] 20
34959 [[2, 6, 4], [1, 3, 7], [8, 0, 5]] 20
34959 [[2, 6, 4], [1, 3, 7], [8, 0, 5]] 20
34960 [[2, 4, 3], [1, 7, 0], [8, 6, 5]] 20
34960 [[2, 4, 3], [1, 7, 0], [8, 6, 5]] 20
34961 [[8, 1, 7], [0, 2, 5], [4, 3, 6]] 20
34961 [[8, 

KeyboardInterrupt: 

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission